In [45]:
from ipynb.fs.full.Background_Functions import *
from ipynb.fs.full._03_Internal_Cells import *

#### Test Case setup: 

The test case used will be a cantilever beam. One end of the beam will be fixed, i.e. the boundary condition is fixed displacement of 0 m. The top and bottom boundaries will have a traction BC of 0, the loaded end has a fixed traction BC in the negative y-direction. 

The solver will be set up so that any combination of fixed displacement or traction boundary conditions can be used. 

In [46]:
# Cantilever Setup 

tr_right_x = 0    #u boundary condition at the right boundary
tr_right_y = - 1e6   #v boundary condition at the right boundary

tr_top_x = 0    #u boundary condition at the top boundary
tr_top_y = 0       #v boundary condition at the top boundary

tr_bottom_x = 0    #u boundary condition at the bottom boundary 
tr_bottom_y = 0  #v boundary condition at the bottom boundary

u_left = 0
v_left = 0

In [47]:
# Set Boundary Conditions:
class BC_settings:

    # Here is where you can change the BC settings

    left = "fixed_displacement"
    right = "traction"
    top = "traction"
    bottom = "traction"

    def __init__(self, boundary):

        if boundary == "l":
            if BC_settings.left == "traction":
                self.traction = True
                self.fixed_displacement = False
            elif BC_settings.left == "fixed_displacement":
                self.fixed_displacement = True
                self.traction = False

        if boundary == "r":
            if BC_settings.right == "traction":
                self.traction = True
                self.fixed_displacement = False
            elif BC_settings.right == "fixed_displacement":
                self.fixed_displacement = True
                self.traction = False

        if boundary == "t":
            if BC_settings.top == "traction":
                self.traction = True
                self.fixed_displacement = False
            elif BC_settings.top == "fixed_displacement":
                self.fixed_displacement = True
                self.traction = False

        if boundary == "b":
            if BC_settings.bottom == "traction":
                self.traction = True
                self.fixed_displacement = False
            elif BC_settings.bottom == "fixed_displacement":
                self.fixed_displacement = True
                self.traction = False

BC_settings("b").fixed_displacement

False

In [48]:
class boundary_U:

    def __init__(self, boundaries, xy):

        if xy == "x":
            if boundaries[0] == "b":
                    self.BC = u_bottom
            if boundaries[0] == "t":
                    self.BC = u_top
            if boundaries[0] == "l":
                    self.BC = u_left
            if boundaries[0] == "r":
                    self.BC = u_right

        if xy == "y":
            if boundaries[0] == "b":
                    self.BC = v_bottom
            if boundaries[0] == "t":
                    self.BC = v_top
            if boundaries[0] == "l":
                    self.BC = v_left
            if boundaries[0] == "r":
                    self.BC = v_right

# Fixed Displacement Boundary Conditions
***

Fundamentally up to this point we have been outlining methods of how to solve for displacement values ($u, v$) that satisfy the momentum equation. For some boundaries it could be the case that the displacement of this boundary is fixed. 

*Can you think of any examples?*

Here's one: 

- The fixed end of a cantilever beam

You might recognise this symbol from your solid mechanics classes: 

<img src="./Paper_Images/fixed_displacement.png" alt="Drawing" style="width: 400px;"/> 

This means that at the "wall" the displacement values don't change with time, i.e. $\boldsymbol{u}$ = 0. 

<img src="./Paper_Images/fixed_displacement_zoomed_in.png" alt="Drawing" style="width: 500px;"/> 


*How do you think this will this effect how we code the solver?*

In essence the points on the boundary (where $\boldsymbol{u}$ is fixed) are simple to code as we are given $\boldsymbol{u}$. However, what's not so simple is how this effects the surrounding cells, specifically the finite-difference approximations. 


## Points on the boundary

These values are known, $a_p$ is set to 1 and $b$ is set to the fixed displacement value. 

For the boundary points we assign the set values as the $b$ term and 1 as the $a_P$ term

$$ [1][u] = [b]$$

$$ u = b$$

In [40]:
def displacement_point_BCs(A_matrix, b_matrix, k, boundaries, xy):
    
    A_matrix[k,k] = 1e10
    b_matrix[k] = boundary_U(boundaries, xy).BC*1e10

    return A_matrix, b_matrix

## $A$-matrix terms

For the boundary cells, the only term that will be affected is the term assosciated with the boundary face, i.e. the $N$ term for the top boundary, the $S$ term for the bottom boundary,  the $E$ term for the right boundary, the $W$ term for the left boundary

We now take treat the point on the boundary the same as how we treated the internal cell centre points (previous points). As a result the denominator distance ($\delta x, \delta y$) decreases by 50 %.

Below shows an example of the top boundary and how this discretisation effects the x and y momentum equations:

<img src="./Paper_Images/boundaryCellDisplacement.png" alt="Drawing" style="width: 600px;"/> 


### x-equation:

$$
\frac{\rho}{\Delta t^2} \left( u_P^n V^n \right) 
-
\mu \left( \dfrac{ u_N - u_P }{dy\color{red}{/2}} \right)
 |S_N|
+
\mu \left( \dfrac{ u_P - u_S }{dy} \right)
  |S_S|
- 
(2\mu + \lambda)
\left(\dfrac{ u_E - u_P}{dx} \right)
  |S_E|
+ 
(2\mu + \lambda)
\left(\dfrac{ u_P - u_W}{dx}\right) 
  |S_W|
= 
RHS
$$


$$
u_P \left[
\frac{\rho}{\Delta t^2} V^n
+ 
\color{red} 2\dfrac{\mu |S_N|}{dy} 
+
\dfrac{\mu |S_S|}{dy} 
+
\dfrac{(2\mu + \lambda) |S_E|}{dx}
+
\dfrac{(2\mu + \lambda) |S_W|}{dx}
\right]
-
\color{red} 2 u_N \dfrac{\mu |S_N|}{dy}
-
u_S \dfrac{\mu |S_S|}{dy}
- 
u_E \dfrac{(2\mu + \lambda) |S_E|}{dx} 
- 
u_W \dfrac{(2\mu + \lambda) |S_W|}{dx} 
= 
RHS
$$

<!-- a_p = ... -->


$$ a_P = \left[
\frac{\rho}{\Delta t^2} V^n
+ 
\color{red}{2}\dfrac{\mu |S_N|}{dy} 
+
\dfrac{\mu |S_S|}{dy} 
+
\dfrac{(2\mu + \lambda) |S_E|}{dx}
+
\dfrac{(2\mu + \lambda) |S_W|}{dx}
\right]  
$$

$$
\begin{align*} 
a_N & = \color{red}{2}\dfrac{\mu |S_N|}{dy} & = & &\color{red}{2}(K_{N, u, x}) \dfrac{|S_N|}{dy} \\
a_S & = \dfrac{\mu |S_S|}{dy} & = & &(-K_{S, u, x}) \dfrac{|S_S|}{dy} \\
a_E & = \dfrac{(2\mu + \lambda) |S_E|}{dx} & = & &(K_{E, u, x}) \dfrac{|S_E|}{dx} \\
a_W & = \dfrac{(2\mu + \lambda) |S_W|}{dx} & = & &(-K_{W, u, x}) \dfrac{|S_W|}{dx} \\
\end{align*} 
$$


### y-equation:

$$
\frac{\rho}{\Delta t^2} \left(   v_p^n V^n \right)
-
(2\mu + \lambda)
\left(\dfrac{v_N - v_P}{dy\color{red}{/2}}\right) 
  |S_N|
+
(2\mu + \lambda)
\left(\dfrac{v_P - v_S}{dy} \right)
 |S_S|
- 
\mu
\left( \dfrac{v_E - v_P}{dx}\right)
  |S_E|
+ 
\mu
\left( \dfrac{v_P - v_W}{dx}\right)
  |S_W|
= 
RHS
$$

$$
v_P \left[
\frac{\rho}{\Delta t^2} V^n
+ 
\color{red} 2 \dfrac{(2\mu + \lambda)  |S_N|}{dy} 
+
\dfrac{(2\mu + \lambda)  |S_S|}{dy} 
+
\dfrac{\mu |S_E|}{dx}
+
\dfrac{\mu |S_W|}{dx}
\right]
-
\color{red} 2 v_N \dfrac{(2\mu + \lambda) |S_N|}{dy}
-
v_S \dfrac{(2\mu + \lambda) |S_S|}{dy}
- 
v_E \dfrac{\mu |S_E|}{dx} 
- 
v_W \dfrac{\mu |S_W|}{dx} 
= 
RHS
$$


$$
a_P  = \left[
\frac{\rho}{\Delta t^2} V^n
+ 
\color{red}{2} \dfrac{(2\mu + \lambda)  |S_N|}{dy} 
+
\dfrac{(2\mu + \lambda)  |S_S|}{dy} 
+
\dfrac{\mu |S_E|}{dx}
+
\dfrac{\mu |S_W|}{dx}
\right]
$$

$$
\begin{align*} 
a_N &=  \color{red}{2} \dfrac{(2\mu + \lambda) |S_N|}{dy} &=&  &\color{red}{2} (K_{N, v, y}) \dfrac{|S_N|}{dy}&  \\
a_S &=  \dfrac{(2\mu + \lambda) |S_S|}{dy} &=& &(-K_{S, v, y}) \dfrac{|S_S|}{dy}&  \\
a_E &=  \dfrac{\mu |S_E|}{dx}  &=&  &(K_{E, v, y}) \dfrac{|S_E|}{dx}& \\
a_W &=  \dfrac{\mu |S_W|}{dx}  &=&  &(-K_{W, v, y}) \dfrac{|S_W|}{dx}& \\
\end{align*}
$$


Essentially, the terms for the A matrix are the same for the internal cells except the terms associated with the boundary face are doubled (*<span style="color:red">outlined in red above</span>*)

This process is the same for each boundary direction $N$, $S$, $E$, $W$

In [49]:
# For cell centres on a fixed displacement boundary:

# Example values 
xy = "x"
boundaries = ["t"]

# Initialise a terms to the same as internal cell values
a_N = A(xy).a_N
a_S = A(xy).a_S
a_E = A(xy).a_E
a_W = A(xy).a_W

# Double a terms if on the boundary
for boundary in boundaries:
    if boundary == "b": a_S = A(xy).a_S*2
    if boundary == "t": a_N = A(xy).a_N*2
    if boundary == "l": a_W = A(xy).a_W*2
    if boundary == "r": a_E = A(xy).a_E*2

# Sum the boundary a terms and the temporal a term for a_p
if transient:
    a_P = (rho*dx*dy/(dt**2)) + a_N + a_S + a_E + a_W
else:
    a_P = a_N + a_S + a_E + a_W

print("Printing example a-term values for fixed displacement cell on the top boundary:\n")
print("a_N should be twice the size of a_S")
print("a_E and a_W should be identical\n")

print("Value for a_N: ", a_N)
print("Value for a_S: ", a_S)
print("Value for a_E: ", a_E)
print("Value for a_W: ", a_W)
print("Value for a_P: ", a_P)

Printing example a-term values for fixed displacement cell on the top boundary:

a_N should be twice the size of a_S
a_E and a_W should be identical

Value for a_N:  307692307692.3077
Value for a_S:  153846153846.15384
Value for a_E:  134615384615.38461
Value for a_W:  134615384615.38461
Value for a_P:  730769230769.2308


This process is the same for each boundary direction $N$, $S$, $E$, $W$

## $b$-matrix terms

For the $b$ term, or RHS of the momentum equation, nothing changes from the internal cells except the approximation of the corner displacement values. 

Instead of using the average of the surrounding cell centre centres, we use the average of the displacements of the face centres either side of the corner on the boundary:

<img src="./Paper_Images/displacementCellDisplacementEdgeCorners.png" alt="Drawing" style="width: 600px;"/> 

In [42]:
def corner(boundaries, corner_placement, uv, U_array, k):
    if uv == "u":
        uv_i = 0
    elif uv == "v":
        uv_i = 1

    # the displacement function returns the u or v values (uv_i) for a particular cell (k) within a displacement field (U_array)
    # disp below is the u or v value at cell k 

    # original corner expression
    if corner_placement == "NE":
        corner =  (1/4)*(disp.P + disp.NE + disp.N + disp.E)
    if corner_placement == "SE":
        corner =  (1/4)*(disp.P + disp.SE + disp.S + disp.E)
    if corner_placement == "SW":
        corner =  (1/4)*(disp.P + disp.SW + disp.S + disp.W)
    if corner_placement == "NW":
        corner =  (1/4)*(disp.P + disp.NW + disp.N + disp.W)

    # update corner value if on the boundary
    for boundary in boundaries:
        if (boundary == "b") & (corner_placement == "SE"):
            corner =  (1/2)*(disp.SE + disp.S)
        if (boundary == "b") & (corner_placement == "SW"):
            corner =  (1/2)*(disp.SW + disp.S)

        if (boundary == "t") & (corner_placement == "NE"):
            corner =  (1/2)*(disp.NE + disp.N)
        if (boundary == "t") & (corner_placement == "NW"):
            corner =  (1/2)*(disp.NW + disp.N)

        if (boundary == "l") & (corner_placement == "NW"):
            corner =  (1/2)*(disp.NW + disp.W)
        if (boundary == "l") & (corner_placement == "SW"):
            corner =  (1/2)*(disp.SW + disp.W)

        if (boundary == "r") & (corner_placement == "NE"):
            corner =  (1/2)*(disp.NE + disp.E)
        if (boundary == "r") & (corner_placement == "SE"):
            corner =  (1/2)*(disp.SE + disp.E)

    return corner

That's all that needs to be changed from the internal cell code to allow for fixed displacement boundary conditions! To summarise:

**Points on the Boundary**: These values are known, $a_p$ is set to 1 and $b$ is set to the fixed displacement value. 

**A-matrix terms for cells on the boundary**: The face terms $a_N$, $a_S$, $a_E$ & $a_W$ are initialised to the internal cell values. The terms relating to the face on the boundary ($a_N$ in the example above) is doubled. $a_p$ is the sum of the face terms and the temporal term (if transient)

**b-matrix terms for cells on the boundary**: The the same process used for the internal cells  is followed. However, the corner displacement approximation is updated to use the adjacent face centres on the boudary.



Now let's create a function that assigns these values to the A and b matrix

In [43]:
# This class will inherit the A class from the previous notebook

class boundaryCellDisplacement(A):

    def __init__(self, edges, xy):

        # Initialise a terms to the same as internal cell values
        self.a_N = A(xy).a_N
        self.a_S = A(xy).a_S
        self.a_E = A(xy).a_E
        self.a_W = A(xy).a_W
        
        # Double a terms if on the boundary
        for edge in edges:
            if edge == "b": self.a_S = A(xy).a_S*2
            if edge == "t": self.a_N = A(xy).a_N*2
            if edge == "l": self.a_W = A(xy).a_W*2
            if edge == "r": self.a_E = A(xy).a_E*2
            
        # Sum the boundary a terms and the temporal a term for a_p
        if transient:
            self.a_P = (rho*dx*dy/(dt**2)) + self.a_N + self.a_S + self.a_E + self.a_W
        else:
            self.a_P = self.a_N + self.a_S + self.a_E + self.a_W

    # Use same process as internal cells
    def b_temp(U_old, U_old_old, k, xy):      
        return A.b_temp(U_old, U_old_old, k, xy)

    # This is the same code as class A however a new corner function is decribed below
    def b_force(edges, k, xy, U_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        N_term =(
                    S_N*A.coef(xy, "N", uv)*(
                    (boundaryCellDisplacement.corner(edges, "NE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "NW", uv, U_previous, k))
                    /dx)
                )
        S_term =(
                    S_S*A.coef(xy, "S", uv)*(
                        (boundaryCellDisplacement.corner(edges, "SE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, U_previous, k))
                        /dx)
                ) 
        E_term =(
                    S_E*A.coef(xy, "E", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SE", uv, U_previous, k))
                        /dy)
                ) 
        W_term =(
                    S_W*A.coef(xy, "W", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NW", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, U_previous, k))
                        /dy)
                ) 

        b_force = (N_term + S_term + E_term + W_term)

        return b_force

    # New corner function for the boundary
    def corner(edges, corner_placement, uv, U_previous, k):

        if uv == "u":
            uv_i = 0
        elif uv == "v":
            uv_i = 1

        disp = displacement(k, U_previous, uv_i)

        if corner_placement == "NE":
            corner =  (1/4)*(disp.P + disp.NE + disp.N + disp.E)
        if corner_placement == "SE":
            corner =  (1/4)*(disp.P + disp.SE + disp.S + disp.E)
        if corner_placement == "SW":
            corner =  (1/4)*(disp.P + disp.SW + disp.S + disp.W)
        if corner_placement == "NW":
            corner =  (1/4)*(disp.P + disp.NW + disp.N + disp.W)

        # Updates the calculation process if the point is on the boundary
        for edge in edges:
            if (edge == "b") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.S)
            if (edge == "b") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.S)

            if (edge == "t") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.N)
            if (edge == "t") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.N)

            if (edge == "l") & (corner_placement == "NW"):
                corner =  (1/2)*(disp.NW + disp.W)
            if (edge == "l") & (corner_placement == "SW"):
                corner =  (1/2)*(disp.SW + disp.W)

            if (edge == "r") & (corner_placement == "NE"):
                corner =  (1/2)*(disp.NE + disp.E)
            if (edge == "r") & (corner_placement == "SE"):
                corner =  (1/2)*(disp.SE + disp.E)

        # This part if for the corner point displacements (WThis will be explained in the next notebook)
        if len(edges) > 1:

            if (edges[0] == "b") & (edges[1] == "l") & (corner_placement == "SW"):
                corner = disp.SW 
            if (edges[0] == "b") & (edges[1] == "r") & (corner_placement == "SE"):
                corner = disp.SE
            if (edges[0] == "t") & (edges[1] == "l") & (corner_placement == "NW"):
                corner = disp.NW
            if (edges[0] == "t") & (edges[1] == "r") & (corner_placement == "NE"):
                corner = disp.NE

        
        return corner

# boundaryCellDisplacement.b_force(["b", "l"], 14, "x", U_previous)

The <code>boundaryCellDisplacement</code> class points to the value for the terms in the matrices. However, it doesn't actually assign them to the matrix. 

The function below <code>displacement_cell_BCs</code> takes in the A-matrix and b-matrix and assigns the values from the <code>boundaryCellDisplacement</code> class to a point <code>k</code> in the mesh.

In [44]:
def displacement_cell_BCs(A_matrix, b_matrix, k, boundaries, xy, U_old, U_old_old, U_previous):

    A_matrix[k,k] = boundaryCellDisplacement(boundaries, xy).a_P
    #an
    A_matrix[k,index(k).n] = - boundaryCellDisplacement(boundaries, xy).a_N        
    #as
    A_matrix[k,index(k).s] = - boundaryCellDisplacement(boundaries, xy).a_S   
    #ae
    A_matrix[k, index(k).e] = - boundaryCellDisplacement(boundaries, xy).a_E
    #aw
    A_matrix[k, index(k).w] = - boundaryCellDisplacement(boundaries, xy).a_W

    b_matrix[k] =(
                boundaryCellDisplacement.b_temp(U_old, U_old_old, k, xy)
                +
                boundaryCellDisplacement.b_force(boundaries, k, xy, U_previous)
            )   

    return A_matrix, b_matrix